## IMDClient package architecture

## TODO:

Make diagrams

Finish analysis section
Test with RISE https://github.com/jupyterlab-contrib/rise

In [3]:
import imdclient

with imdclient.IMDClient("localhost", 8889, n_atoms=50786) as client:
    info = client.get_imdsessioninfo()
    print(info)

    frame = client.get_imdframe()
    print(f"Simulation integration step: {frame.step}")
    print(f"Simulation time (fs): {frame.time}")
    print(f"First atom's position (angstroms): {frame.positions[0]}")

IMDSessionInfo(version=3, endianness='<', wrapped_coords=True, energies=False, time=True, box=True, positions=True, velocities=True, forces=True)
Simulation integration step: 10
Simulation time (fs): 0.01
First atom's position (angstroms): [44.03448  29.363623 30.971706]


## Iterating through a trajectory

In [ ]:
import imdclient

with imdclient.IMDClient("localhost", 8889, n_atoms=50786) as client:
    while True:
        try:
            frame = client.get_imdframe()
            # Do something with the frame
        except EOFError:
            break

## Options

In [ ]:
import imdclient

client = imdclient.IMDClient("localhost", 8889,
                              n_atoms=50786, 
                              # Wait up to 10 seconds for a simulation frame
                              timeout=10,
                              # 1 MB
                              buffer_size=1024 ** 2)

## Software architecture

## Automatic pausing and resuming

In [ ]:
import imdclient

# 2MB Buffer
with imdclient.IMDClient("localhost", 8889, n_atoms=50786, buffer_size=2 * 1024**2) as client:
    while True:
        try:
            frame = client.get_imdframe()
        except EOFError:
            break

## Reader wraps client, handles its limitations

A stream can only be read once. This is a limitation of the client. The reader wraps the client and handles this limitation.

In [10]:
from imdclient.IMD import IMDReader
import MDAnalysis as mda

u = mda.Universe("sample_simulation/imdgroup.gro", "imd://localhost:8889")

for ts in u.trajectory[:]:
    pass

try:
    for ts in u.trajectory[:]:
        pass
except RuntimeError:
    print("A stream is not a file!")

A stream is not a file!


## API interactions

## Stepping is allowed, but specifying an end frame isn't

This is allowed
```python
for ts in u.trajectory[::10]:
    pass
```

This is not
```python
for ts in u.trajectory[:10]:
    pass
```

The length of the trajectory is not known until it is iterated through.

```python
# Not allowed!
len(u.trajectory)
```

## The client works out of the box with MDAnalysis analysis classes

The analysis class must be able to handle a trajectory without a known length.

In [ ]:
from imdclient.IMD import IMDReader
import MDAnalysis as mda
from MDAnalysis.analysis.rms import RMSF

u = mda.Universe("sample_simulation/imdgroup.gro", "imd://localhost:8889")

imd_rmsf = RMSF(u.atoms).run()

print(imd_rmsf.rmsf)

## Running multiple analysis classes on the same stream

We showed you examples earlier that look like this:

## But you can also do this:

In [ ]:
from imdclient.IMDREADER import IMDReader
import MDAnalysis as mda
from MDAnalysis.analysis.rms import RMSF

r1 = RMSF(u.atoms)
r2 = RMSF(u.atoms)
imdclient.StackableAnalysis(u.trajectory, [r1, r2]).run()